In [1]:
import math
import nltk
import re
import all_data_handler

users = all_data_handler.UserData()

In [2]:
def tag_object(original):
	tags = []
	#Matches robots, robot, bot, bots, etc. 
	robots = re.compile("bot|group|swarm|orange|red", re.I)
	crate = re.compile("crate", re.I)
	targetA = re.compile("area a|box a| a$|to a,|^a$", re.I)
	targetB = re.compile("area b|box b| b$|to b,|^b$", re.I)
	whitespace = re.compile("whitespace|ground|screen|white area", re.I)
	
	toCheck = [(robots, "r"), (crate, "c"), (targetA, "a"), (targetB, "b"), (whitespace, "w")]
	for compiled, tag in toCheck:
		if re.search(compiled, original):
			tags.append(tag)
	return tags


In [3]:
#From https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance#Python
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [4]:
all_task_strs = []
for participant in users.data.keys():
		for task in users.data[participant]["tasks"].keys():
			#We're building a string representation of the user's action on the task
			# Types           Targets
			#-------------    -------------
			# drag 		 D    whitespace  w
			# draw       W    robots      r
			# ui         U    crate       c
			# tap        T    area a      a
			# doubletap  2    area b      b
			# tripletap  3    unclear     _
			# hold       H
			# pinch      P
			# rev pinch  R
			# lasso      L
			# box select B
			# voice      V
			# other      O

			task_str = ""

			for event in users.data[participant]["tasks"][task]:
				#Skip memos and examples
				if event["event_type"] == "memo" or event["example"]:
					continue

				#Get objects for most events
				if "objects" in event.keys():
					#Convert the event type to a code
					eType = event["event_type"]
					if eType == "ui":
						task_str += "U"
					elif eType == "other":
						task_str += "O"
					elif eType == "drag" and event["draw"] is not None:
						task_str += "W"
					elif eType == "drag" and event["draw"] is None:
						task_str += "D"
					elif eType == "pinch" and event["reverse"]:
						task_str += "R"
					elif eType == "pinch" and not event["reverse"]:
						task_str += "P"
					elif eType == "tap" and event["count"] > 1:
						task_str += str(event["count"])
					elif eType == "tap" and event["count"] == 1:
						task_str += "T"
					elif eType == "tap" and event["hold"]:
						task_str += "H"
					elif eType == "lasso":
						task_str += "L"
					elif eType == "box_select":
						task_str += "B"
					
					#Now add the tag objects
					obj = " ".join(event["objects"])
					obj = obj.replace("\"", "")
					obj = obj.lower()
					tags = tag_object(obj)
					if len(tags) > 0:
						task_str += "".join(tags)
					else:
						task_str += "_"

				elif event["event_type"] == "voice_command":
					task_str += "V"

					obj = " ".join(event["command"])
					obj = obj.replace("\"", "")
					obj = obj.lower()
					tags = tag_object(obj)
					if len(tags) > 0:
						task_str += "".join(tags)
					else:
						task_str += "_"
					
			taskName = users.taskNumberToName(int(task), participant)
					
			all_task_strs.append((participant, task, taskName, task_str))

In [5]:
#Import the data from the hand-coding of the strategies, so I have an idea of my ground truth
import csv
new_task_strs = []
with open('command_strategies.csv', 'rb') as csvfile:
    r = csv.reader(csvfile, delimiter=',')
    tasks = r.next()
    #For each row of the csv file, which describes all of a user's strategies
    for row in r:
        user = str(row[0])
        #For every task that that user did
        for index, task in enumerate(tasks[2:]):
            #Find the corresponding task string and add it to the new task strs
            for tstr in all_task_strs:
                if tstr[0] == user and tstr[2] == task:
                    #import pdb; pdb.set_trace()
                    temp = (tstr[0], tstr[1], tstr[2], tstr[3], row[index+2])
                    new_task_strs.append(temp)

In [6]:
#Set up a distance lookup table. This might take a while
# dlt = {}
# for t1 in new_task_strs:
#     dlt[t1] = {}
#     for t2 in new_task_strs:
#         #Only look within tasks
#         if t2[1] == t1[1]:
#             dlt[t1][t2] = levenshtein(t1[3], t2[3])

In [7]:
#get a distance array that scipi can deal with and a list of data it can be registered against
# data_list = []
# distance_list = []
# for pair1 in dlt.keys():
#     for pair2 in dlt[pair1].keys():
#         data_list.append((pair1, pair2))
#         distance_list.append(dlt[pair1][pair2])

In [8]:
#Build a distance lookup that is just upper triangular, rather than all pairs
import itertools

combi_dist = {}
all_pairs = itertools.combinations(new_task_strs, 2)
for pair in all_pairs:
    #Only look at pairs where the task matches
    if pair[0][1] == pair[1][1]:
        if pair[0] not in combi_dist.keys():
            combi_dist[pair[0]] = {}    
        combi_dist[pair[0]][pair[1]] = levenshtein(pair[0][3], pair[1][3])
        
        

In [18]:
results = []
for task in combi_dist.keys():
    #Get the ground truth value
    truth = task[4]
    #Get the k nearest neighbors, starting with neighbors
    d = 0
    neighbors = []
    k = 5
    
    #Get the k nearest neighbors, starting at 0 and going up (levenshtein gives integer distance)
    while len(neighbors) < k:
        for canidate in combi_dist[task]:
            if combi_dist[task][canidate] == d:
                neighbors.append((canidate, d))
                #Could have enough neighbors at this distance to go over k
                if len(neighbors) == k:
                    break
        #Next furthest neighbors
        d += 1
        #if distance ends up exceeding furthest neighbor, quit
        if d > max(combi_dist[task].values()):
            break
    
    #For the neighbors, count the strategies, and average the distance
    counts = {}
    avg_d = 0
    for n, distance in neighbors:
        if n[4] in counts.keys():
            counts[n[4]] += 1
        else:
            counts[n[4]] = 1
        avg_d += float(distance)
    avg_d = avg_d/float(len(neighbors))
    
    #Get the predicted value
    guess = max(counts, key=counts.get)
    results.append((truth, guess, avg_d))
    
    

In [19]:
for result in results:
    print "{}|{}|{}".format(result[0],result[1],result[2])

select/drag|drag robot|10.0
drag|drag|0.6
drag|drag|2.0
drag robot|drag robot|8.2
2x select/move|drag|2.8
border|to & around|1.8
border|border|1.2
draw wall|doubletap whitespace|1.2
drag|drag|0.4
2x select/tap|drag|7.2
rev pinch|squiggle|2.0
select/drag crate|drag crate|4.8
position/move|position/move|13.6
cut|cut|0.0
tap|tap|0.4
2x select/drag|2x select/drag|4.4
2x select/drag|2x select/drag|0.6
position, no move|position/move|9.6
cut/drag|2x select/drag|14.0
position/move|position/move|3.2
draw line behind robots|tap robot|1.2
drag back|doubletap robot|1.0
select/ui|tap|0.0
2x select/drag|drag|3.8
drag|drag|0.0
other robots push offscreen|drag offscreen|9.2
drag crate|drag crate|10.5
drag robot|drag robot|0.0
doubletap|select/ui|0.8
sweep around screen|select/drag around|2.0
select/drag around|border|1.2
drag|drag|3.2
voice |drag|5.2
border|border|1.6
ui remove|edge|4.6
corner|pinch|0.6
around screen|border|1.2
2x select/move|drag|3.0
ui delete|drag offscreen|1.0
doubletap robot|tap 

In [22]:
#Get the counts of strings for each strategy
strByStrat = {}
for t in new_task_strs:
    if t[4] not in strByStrat.keys():
        strByStrat[t[4]] = {}
    if t[3] in strByStrat[t[4]].keys():
        strByStrat[t[4]][t[3]] += 1
    else:
        strByStrat[t[4]][t[3]] = 1

In [25]:
for strategy in strByStrat.keys():
    print strategy
    print "="*len(strategy)
    for k, v in sorted(strByStrat[strategy].iteritems(), key= lambda (k,v): (v,k)):
        print "{}\t{}".format(v, k)
    print ""

select/rev pinch
3	BrRr

ui
==
1	

draw, drag to line
1	WwWrw

ui, select
1	DwLr

flat hand robots
1	OrTr

2x select/move
1	BrBrDrDr
1	BrBrDraBrDbw
1	BrBrDrbBrDra
1	BrDraBrDra
1	BrTbBrDrbBrDra
1	BrTwDbwBrTwTa
1	LrDrDrbLrDrbLrTaDra
1	LrDrbDaLrDrb
1	LrDrbLrDr
1	LrTaLrTb
1	LrWraLrWrb
1	LrWrbLrWra
1	RrDrb
1	TrWrbTrWra
1	TwTwWraWrb
2	BrDrbBrDra
2	LrDrbLrDra
3	Vrab
5	LrDraLrDrb

corner
1	2rWr
1	BrTrDr
1	LrDr
1	LrDrLrDrTrLrDr
1	TrDrT_Or3r
1	TrTwTrTrTwTrTrwLrDr
1	Wr
3	Dr

move away
1	DrOr
1	TrTwTwTw

select/tap
1	BrBrBrTrwTwBr
1	BrTa
1	BrTw
1	LrTa
1	LrTr
1	LrTw

chop & move
1	OrOrOr

lasso
=====
3	Lr

cut/drag
1	WrDrWrDrTrTrTrWrWrTr

ui select
1	

select/drag crate
1	2cWca
1	2rDa
1	WcDcaTa

draw "X"
1	DrDr
1	LrDrDr
1	Wr
1	WrWr

rasied hand pull back
1	Ow

position, no move
1	BrDrcTc
1	DrcDrcDrcDrcDrc

r pinch/drag
1	Orab

multi pinch
1	PrPrPrPrPr

2h drag
1	DraDrbDraDrb
1	DrbDra

select/redirect
1	BrTw
1	LrDr

multiple push
1	OrObOrOa

offscreen garbage bin
1	Wrw

select robots
1	Br

draw "P->

In [47]:
import re

def condenseString(inStr):
    #Replace "B" (box select), and "L" (lasso select) with "S" (general select)
    inStr = inStr.replace("B", "S")
    inStr = inStr.replace("L", "S")
    #Taps, doubletaps, tripletaps on robots or crate are selections too
    inStr = inStr.replace("Tr", "Sr")
    inStr = inStr.replace("2r", "Sr")
    inStr = inStr.replace("3r", "Sr")
    inStr = inStr.replace("Tc", "Sc")
    inStr = inStr.replace("2c", "Sc")
    inStr = inStr.replace("3c", "Sc")
    #Multiple selections in a row are just the last selection
    inStr = re.sub(r"S([rawcb_]+)(S\1)*", r"S\1", inStr)
    return inStr

print condenseString("DrDraDraTrLrDraLrDraTa")
print condenseString("TrTw2rTw2rTw")
print condenseString("BrBrDrDrBrBrTr")
print condenseString("BrBrDrwDw")

DrDraDraSrDraSrDraTa
SrTwSrTwSrTw
SrDrDrSr
SrDrwDw


In [48]:
print new_task_strs[0]

(u'1', u'13', 'crate_dispersed', '2cDrca', 'position/move')


In [49]:
condensed = []
for t in new_task_strs:
    condensed.append((t[0], t[1], t[2], t[3], condenseString(t[3]), t[4]))

In [51]:
#Get the counts of strings for each strategy
strByStrat = {}
for t in condensed:
    if t[5] not in strByStrat.keys():
        strByStrat[t[5]] = {}
    if t[4] in strByStrat[t[5]].keys():
        strByStrat[t[5]][t[4]] += 1
    else:
        strByStrat[t[5]][t[4]] = 1

In [52]:
for strategy in strByStrat.keys():
    print strategy
    print "="*len(strategy)
    for k, v in sorted(strByStrat[strategy].iteritems(), key= lambda (k,v): (v,k)):
        print "{}\t{}".format(v, k)
    print ""

select/rev pinch
3	SrRr

ui
==
1	

draw, drag to line
1	WwWrw

ui, select
1	DwSr

flat hand robots
1	OrSr

2x select/move
1	RrDrb
1	SrDrDr
1	SrDrDrbSrDrbSrTaDra
1	SrDraSrDbw
1	SrDraSrDra
1	SrDrbDaSrDrb
1	SrDrbSrDr
1	SrTaSrTb
1	SrTbSrDrbSrDra
1	SrTwDbwSrTwTa
1	SrWraSrWrb
1	TwTwWraWrb
2	SrWrbSrWra
3	Vrab
5	SrDraSrDrb
5	SrDrbSrDra

corner
1	SrDrSrDrSrDr
1	SrDrT_OrSr
1	SrTwSrTwSrwSrDr
1	SrWr
1	Wr
2	SrDr
3	Dr

move away
1	DrOr
1	SrTwTwTw

select/tap
1	Sr
1	SrwTwSr
2	SrTa
2	SrTw

chop & move
1	OrOrOr

lasso
=====
3	Sr

cut/drag
1	WrDrWrDrSrWrWrSr

ui select
1	

select/drag crate
1	ScWca
1	SrDa
1	WcDcaTa

draw "X"
1	DrDr
1	SrDrDr
1	Wr
1	WrWr

rasied hand pull back
1	Ow

position, no move
1	DrcDrcDrcDrcDrc
1	SrDrcSc

r pinch/drag
1	Orab

multi pinch
1	PrPrPrPrPr

2h drag
1	DraDrbDraDrb
1	DrbDra

select/redirect
1	SrDr
1	SrTw

multiple push
1	OrObOrOa

offscreen garbage bin
1	Wrw

select robots
1	Sr

draw "P->A"
1	WwWwWwWwWwWw

select/r pinch
1	SrRr
1	SrWrRr

select/4f scatter
1	SrOr
1	SrOrOr

